In [6]:
from fuzzywuzzy import fuzz
city_circuit_mapping = {
  "Aachen": "549",
  "Amberg": "157",
  "Ansbach": "162",
  "Arnsberg": "470",
  "Aschaffenburg": "187",
  "Augsburg": "111",
  "Aurich": "418",
  "Bad Hersfeld": "290",
  "Bad Homburg v.d.H.": "282",
  "Bad Kreuznach": "572",
  "Bad Oeynhausen": "487",
  "Bamberg": "190",
  "Bayreuth": "194",
  "Berlin": "217",
  "Berlin (Charlottenburg)": "217",
  "Bielefeld": "480",
  "Bochum": "490",
  "Bonn": "557",
  "Braunschweig": "360",
  "Bremen": "258",
  "Chemnitz": "634",
  "Coburg": "197",
  "Coesfeld": "530",
  "Cottbus": "228",
  "Darmstadt": "270",
  "Deggendorf": "117",
  "Dortmund": "499",
  "Dresden": "621",
  "Duisburg": "446",
  "Düren": "550",
  "Düsseldorf": "441",
  "Eschwege": "305",
  "Essen": "507",
  "Flensburg": "695",
  "Frankfurt am Main": "281",
  "Frankfurt/Oder": "237",
  "Freiburg": "8",
  "Friedberg": "296",
  "Fritzlar": "306",
  "Fulda": "286",
  "Fürth": "167",
  "Gelsenkirchen": "510",
  "Gießen": "297",
  "Göttingen": "372",
  "Gütersloh": "482",
  "Hagen": "516",
  "Hamburg": "261",
  "Hamm": "500",
  "Hanau": "300",
  "Hannover": "379",
  "Hildesheim": "387",
  "Hof": "200",
  "Homburg": "654",
  "Ingolstadt": "212",
  "Iserlohn": "517",
  "Jena": "739",
  "Kaiserslautern": "609",
  "Kassel": "308",
  "Kempten": "121",
  "Kempten (Allgäu)": "121",
  "Kerpen": "567",
  "Kiel": "706",
  "Kleve": "454",
  "Koblenz": "582",
  "Köln": "568",
  "Königstein": "283",
  "Korbach": "309",
  "Krefeld": "458",
  "Landau": "615",
  "Landshut": "127",
  "Langenfeld": "444",
  "Lebach": "655",
  "Leipzig": "649",
  "Lemgo": "497",
  "Limburg": "320",
  "Lübeck": "716",
  "Ludwigshafen": "606",
  "Ludwigshafen a.Rhein (Ludwigshafen)": "606",
  "Lüneburg": "394",
  "Mainz": "593",
  "Mannheim": "33",
  "Marburg": "328",
  "Memmingen": "134",
  "Merzig": "656",
  "Mönchengladbach": "462",
  "Montabaur": "586",
  "München": "136",
  "Münster": "535",
  "Neubrandenburg": "339",
  "Neunkirchen": "657",
  "Neuruppin": "249",
  "Neuss": "442",
  "Nürnberg": "172",
  "Offenbach": "279",
  "Offenbach am Main": "279",
  "Oldenburg": "428",
  "Oldenburg (Oldenburg)": "428",
  "Osnabrück": "439",
  "Ottweiler": "658",
  "Paderborn": "543",
  "Passau": "146",
  "Pinneberg": "703",
  "Potsdam": "252",
  "Recklinghausen": "492",
  "Regensburg": "177",
  "Rostock": "349",
  "Saarbrücken": "659",
  "Saarlouis": "660",
  "Schleiden": "553",
  "Schweinfurt": "207",
  "Schwerin": "358",
  "Siegburg": "561",
  "Siegen": "548",
  "St. Ingbert": "661",
  "St. Ingbert (St Ingbert)": "661",
  "St. Wendel": "662",
  "St. Wendel (St Wendel)": "662",
  "Stadthagen": "368",
  "Steinfurt": "529",
  "Stendal": "691",
  "Stralsund": "351",
  "Straubing": "179",
  "Stuttgart": "95",
  "Tostedt": "404",
  "Traunstein": "155",
  "Ulm": "109",
  "Völklingen": "664",
  "Walsrode": "417",
  "Weiden": "185",
  "Weiden i. d. OPf.": "185",
  "Wetzlar": "322",
  "Wiesbaden": "335",
  "Wittlich": "602",
  "Wuppertal": "469",
  "Würzburg": "211",
  "Zweibrücken": "618"
}

# Todo: throw error if match too big
def map_city_to_circuitId(city: str) -> str:
  closest_key = max(city_circuit_mapping.keys(), key=lambda x: fuzz.ratio(city, x))
  return city_circuit_mapping[closest_key]

In [31]:
from supabase import create_client, Client

class DocumentManager:
    def __init__(self, supabase_url: str, supabase_key: str):
        self.supabase: Client = create_client(supabase_url, supabase_key)

    def _save_document_link_to_db(self, full_path: str, company_id: int):
        # Define the table name where you want to save the document link
        print("we are in the function now")
        table_name = 'startups'

        # Create a new record or update existing with the company_id and the full_path
        data = {'link_SI_file_current': full_path}
        print("we now atttemot the update")
        # Insert or update the data into the table
        response = self.supabase.table(table_name).update(data).eq('startup_id', company_id).execute()

        # Check if the operation was successful
        # if response. in range(200, 300):
        #     print("Document link saved successfully.")
        #     return response.data
        # else:
        #     # Handle any errors that occur during the insert
        #     error_message = response.get('error', {}).get('message', 'Unknown error')
        #     print(f"Failed to save document link: {error_message}")
        #     return None
import os
# Usage example:
# You need to replace 'your_supabase_url' and 'your_supabase_key' with the actual values
document_manager = DocumentManager(os.getenv('SUPABASE_URL'), os.getenv('SUPABASE_KEY'))
document_manager._save_document_link_to_db(full_path="Tacto Technology GmbH_München_HRB 257852/Liste der Gesellschafter - Aufnahme in den Registerordner am 18.11.2022-Tacto Technology GmbH.pdf", company_id=5)

we are in the function now
we now atttemot the update


2024-02-21 12:56:29,125:INFO - HTTP Request: PATCH https://vzaliuejopysgkdewpip.supabase.co/rest/v1/startups?startup_id=eq.5 "HTTP/1.1 200 OK"
